In [1]:
import toml
import torch
import random
import lightning.pytorch as pl
from toolz import get_in
from pathlib import Path
from aging.size_norm import models
from toolz import dissoc, merge, valfilter, assoc, get_in
from aging.size_norm.data import AugmentationParams, TrainingPaths
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from aging.size_norm.lightning import SizeNormModel, ImageDisplay, BehaviorValidation

In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
config = toml.load('/home/wg41/code/ontogeny/configs/00-sizenorm_training_template.toml')

In [4]:
paths = TrainingPaths(**dissoc(config['paths'], 'saving'))
checkpoint = None
save_folder = Path(config['paths']['saving'])
save_folder.mkdir(exist_ok=True, parents=True)

In [5]:
config['augmentation']['rng'] = random.Random(config['augmentation']['seed'])

In [6]:
pl.seed_everything(config['augmentation']['seed'])

[rank: 0] Global seed set to 0


0

In [7]:
augmentation = AugmentationParams(**dissoc(config['augmentation'], 'seed'))

In [8]:
match get_in(["model", "lightning", "arch"], config):
    case 'ae':
        model_arch = models.Autoencoder
    case 'unet':
        model_arch = models.UNet
    case 'efficient':
        model_arch = models.EfficientAutoencoder
    case 'efficient_unet':
        model_arch = models.EfficientUNet
    case _:
        raise ValueError(f"Unknown model architecture {config['model']['arch']}")

In [9]:
model_params = merge(config['model'], config['model'][config['model']['lightning']['arch']])
model_params = valfilter(lambda x: not isinstance(x, dict), model_params)
if 'activation' in model_params:
    model_params = assoc(model_params, 'activation', getattr(torch.nn, model_params['activation']))

In [10]:
model = SizeNormModel(
    paths,
    augmentation,
    model=model_arch,
    model_params=model_params,
    batch_size=64 if get_in(['model', 'init_channel'], config, 32) < 512 else 32,
    **dissoc(config['model']['lightning'], 'arch'),
    seed=config['augmentation']['seed'],
)

In [11]:
ckpt_cb = ModelCheckpoint(
    save_folder,
    filename=f"{model_arch.__name__}" + "-{epoch:02d}-{val_loss:.2e}",
    monitor="val_loss",
    save_top_k=1,
    mode="min",
)

early_stopping_cb = EarlyStopping(
    monitor="val_loss",
    patience=get_in(['callbacks', 'stopping', 'patience'], config, 13),
    mode="min",
    verbose=True,
)

dynamics_cb = BehaviorValidation(
    paths.validation,
    log_interval=get_in(
        ["callbacks", "dynamics_validation", "log_interval"], config, 30
    ),
    max_frames=get_in(
        ["callbacks", "dynamics_validation", "max_frames"], config, 8000
    ),
    validation_type="dynamics",
)

In [ ]:
image_cb = ImageDisplay(log_interval=75)

trainer = pl.Trainer(
    max_epochs=config.get('trainer', {}).get('max_epochs', 85),
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    # callbacks=[ckpt_cb, early_stopping_cb, image_cb, dynamics_cb],
    callbacks=[ckpt_cb, early_stopping_cb, image_cb],
    precision='16-mixed' if torch.cuda.is_available() else "bf16-mixed",
    logger=[CSVLogger(save_folder, name="size_norm_scan"), TensorBoardLogger(save_folder, name="size_norm_scan")],
)
trainer.fit(model, ckpt_path=checkpoint)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wg41/miniconda3/envs/aging/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /n/groups/datta/win/longtogeny/size_norm/models/warp_test-2023-09-30_v00 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | Autoencoder | 12.6 M
--------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
50.473    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

/home/wg41/miniconda3/envs/aging/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/wg41/miniconda3/envs/aging/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x08\x00\x00\x00\x08\x08\x00\x00\x00\x00\xe1d\xe1…

...

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.001


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.001


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
if not trainer.interrupted:
    model = SizeNormModel.load_from_checkpoint(ckpt_cb.best_model_path)
    mdl = torch.jit.trace(
        model.model.eval(),
        torch.zeros(
            model.hparams.batch_size,
            1,
            model.hparams.image_dim,
            model.hparams.image_dim,
            device=model.device,
        ),
    )
    torch.jit.save(mdl, save_folder / "model.pt")

In [ ]:
save_folder